In [1]:
!pip install tokenizers

     |████████████████████████████████| 3.3 MB 5.4 MB/s 


In [26]:
! pip install transformers

     |████████████████████████████████| 3.1 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 35.7 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip install datasets

     |████████████████████████████████| 290 kB 5.3 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 132 kB 47.4 MB/s 
     |████████████████████████████████| 243 kB 43.5 MB/s 
     |████████████████████████████████| 1.1 MB 38.5 MB/s 
     |████████████████████████████████| 160 kB 47.4 MB/s 
     |████████████████████████████████| 271 kB 47.0 MB/s 
     |████████████████████████████████| 192 kB 46.8 MB/s 


In [5]:
from tokenizers import ByteLevelBPETokenizer
from datasets import load_dataset


In [7]:
#load the wiki-text dataset to be used for the training
dataset = load_dataset('wikitext', 'wikitext-103-raw-v1')

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/192M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [10]:
len(dataset['train']),len(dataset['test']),len(dataset['validation'])

(1801350, 4358, 3760)

In [11]:
dataset['train'],dataset['test'],dataset['validation']

(Dataset({
     features: ['text'],
     num_rows: 1801350
 }), Dataset({
     features: ['text'],
     num_rows: 4358
 }), Dataset({
     features: ['text'],
     num_rows: 3760
 }))

In [14]:
dataset['train'].features,dataset['train'][10]

({'text': Value(dtype='string', id=None)},
 {'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictate

In [15]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 10_000:
        # once we git the 10K mark, save to file
        with open(f'/content/drive/MyDrive/byteleveltokenizers/wikitext/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 10K chunks, we will have ~2082 leftover samples, we save those now too
with open(f'/content/drive/MyDrive/byteleveltokenizers/wikitext/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/1801350 [00:00<?, ?it/s]

In [16]:
from pathlib import Path

paths = [str(x) for x in Path('/content/drive/MyDrive/byteleveltokenizers/wikitext').glob('*.txt')]

paths[-5:]

['/content/drive/MyDrive/byteleveltokenizers/wikitext/text_176.txt',
 '/content/drive/MyDrive/byteleveltokenizers/wikitext/text_177.txt',
 '/content/drive/MyDrive/byteleveltokenizers/wikitext/text_178.txt',
 '/content/drive/MyDrive/byteleveltokenizers/wikitext/text_179.txt',
 '/content/drive/MyDrive/byteleveltokenizers/wikitext/text_180.txt']

In [17]:
# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

In [18]:
# Customize training
tokenizer.train(files=paths, vocab_size=30_522, min_frequency=1, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
], show_progress: True)

In [23]:
# Get sequence length max of 256
tokenizer.enable_truncation(max_length=256)

In [29]:
# Save files to disk
tokenizer.save_model("/content/drive/MyDrive/byteleveltokenizers/pretrained_model")

['/content/drive/MyDrive/byteleveltokenizers/pretrained_model/vocab.json',
 '/content/drive/MyDrive/byteleveltokenizers/pretrained_model/merges.txt']

In [32]:
from transformers import RobertaTokenizer
#you will add a confug.json file and write inside it the type of the model {"model_type": "Roberta"}
tokenizer = RobertaTokenizer.from_pretrained("/content/drive/MyDrive/byteleveltokenizers/pretrained_model", max_len=256)

In [33]:
# test our tokenizer on a simple sentence
tokens = tokenizer('ciao, come va?')

In [34]:
print(tokens)

{'input_ids': [0, 71, 20240, 16, 2837, 410, 69, 35, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
